In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt  
import plotly.plotly as py
import plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.tools as tls
init_notebook_mode(connected=True) #plotly offline
from scipy.io import arff
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## Abrir achivo em diferentes formatos

In [ ]:
#Abrir un arhivo formato csv
# df = pd.read_csv(filepath_or_buffer='Arffs/java_train.csv',delimiter='\t')
# df = pd.read_csv(filepath_or_buffer='Arffs/thre=0123451.csv', delimiter='\t')

# Abrir un archivo formato arff 
# filename = "thre=0123451.arff"
# data = arff.loadarff(filename)
# df = pd.DataFrame(data[0])
# df.head()



# train = pd.read_csv(filepath_or_buffer='Arffs/java_train.csv', delimiter='\t')
# test = pd.read_csv(filepath_or_buffer='Arffs/java_test.csv', delimiter='\t')
df = pd.read_csv(filepath_or_buffer='Arffs/java_todo.csv', delimiter='\t')


### Divide data into train and test

After train_test_split

You do that on the training set of data. But then you have to apply the same transformation to your testing set (e.g. in cross-validation), or to newly obtained examples before forecast. But you have to use the same two parameters μ and σ (values) that you used for centering the training set.

These methods are used to center/feature scale the given data. It basically helps to normalize the data within a particular range

For this, we use Z-score method.

Z-Score

We do this on the training set of data.

1.Fit(): Method calculates the parameters μ and σ and saves them as internal objects.

2.Transform(): Method using these calculated parameters apply the transformation to a particular dataset.

3.Fit_transform(): joins the fit() and transform() method for transformation of dataset.

Code snippet for Feature Scaling/Standardisation(after train_test_split).

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit_tranform(X_train)
sc.tranform(X_test)
We apply the same(training set same two parameters μ and σ (values)) parameter transformation on our testing set.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# X = pcadf.iloc[:, 0:35]
# y = pcadf['class']
X_todo = df.iloc[:, 0:35]
y_todo = df[['paciente','name_class','class']]
X_train, X_test, y_train, y_test = train_test_split(X_todo, y_todo, random_state=0)


# print(y) 
#You do that on the training set of data. But then you have to apply the same transformation to your testing set (e.g. in cross-validation), or to newly obtained examples before forecast. But you have to use the same two parameters μ and σ (values) that you used for centering the training set.
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 
X_todo = MinMaxScaler().fit_transform(X_todo) 
print(type(X_train)) #<class 'numpy.ndarray'>
print(type(X_test)) #<class 'numpy.ndarray'>
print(type(y_train)) #<class 'pandas.core.frame.DataFrame'>
print(type(y_test)) #<class 'pandas.core.frame.DataFrame'>
# print(X_train.shape)#(157, 35)


## PCA

In [ ]:
# x = df.iloc[:, 0:35].values #
# column = ['PCA'+str(i) for i in range(1,36)]
# y= df.loc[:,['class']].values
# x = StandardScaler().fit_transform(x)


# pca = PCA(n_components=35)
# principalComponents = pca.fit_transform(x)
# principalDf = pd.DataFrame(data = principalComponents, columns = column)

# pcadf = pd.concat([principalDf, df[['paciente']],df[['name_class']],df[['class']]], axis = 1)
# pcadf.head()

stdscaler = StandardScaler() 

xtrain = stdscaler.fit_transform(X_train)
xtest = stdscaler.transform(X_test)
xtodo = StandardScaler().fit_transform(X_todo)


y_dftrain = pd.DataFrame(data = (y_train.iloc[:,0:3]).values, columns = ['paciente','name_class', 'class'])
y_dftest = pd.DataFrame(data = (y_test.iloc[:,0:3]).values, columns = ['paciente','name_class', 'class'])
 

pca = PCA(n_components=35)
pca_train = pca.fit_transform(xtrain)
pca_test = pca.transform(xtest)
pca_todo = PCA(n_components=35).fit_transform(xtodo)

colunas= ['PCA'+str(i) for i in range(1,36)]

principalDf_train = pd.DataFrame(data = pca_train, columns =colunas) #(157, 35)
principalDf_test = pd.DataFrame(data = pca_test, columns = colunas) #(157, 35)
principalDf_todo = pd.DataFrame(data = pca_todo, columns = colunas) #(157, 35)
 
pca_todo = pd.concat([principalDf_todo, y_todo[['paciente']],y_todo[['name_class']],y_todo[['class']]], axis = 1) 
print(pca_train.shape)#(157, 38)
print(pca_test.shape)#(53, 38)
print(pca_todo.shape)#(210, 38)
pca_todo



### Save in diferent formats



In [ ]:
#save pca to arff
pcadf_save = pd.concat([principalDf,df[['name_class']]], axis = 1)
import arff 
arff.dump('pca_javabest.arff', pcadf_save.values, relation='pca_javabest', names=pcadf_save.columns)

#save to csv
# df = pd.concat([data, col1['paciente'], col2['name_class'],col3['class']], axis = 1)
# df.to_csv('Arffs/thre=0123451.csv', sep='\t', index=False)

### Plotly

In [ ]:
gb = pca_todo.groupby('name_class')
groups = dict(list(gb))
group = set(pca_todo['name_class'])


data = [go.Scatter(x=groups[g].PCA1, y=groups[g].PCA2,
                   mode='markers', text= groups[g].name_class+" "+groups[g].paciente, name=g) 
        for g in group ]

layout = go.Layout(title='', xaxis={'title': 'PCA1'}, yaxis={'title': 'PCA2'}, width=700,height=700)




fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='pca1_pca2')

## TSNE

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
 
x = df.iloc[:, 0:35].values #
y= df.loc[:,['class']].values

X_embedded = TSNE(n_components=2).fit_transform(x)
X_embedded.shape


## NO- dimensionality reduction

In [ ]:
x = df.iloc[:, 0:35].values #
y= df.loc[:,['class']].values
x = StandardScaler().fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# X = pcadf[['PCA1', 'PCA2']]
# y = pcadf['class']
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


def plot_boundary(X, y, clf):
#     X_mat = X[['PCA1','PCA2']].values
    X_mat = X[:, (0,1)] 
    y_mat = y.values

    # Create color maps
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA'])
    cmap_bold  = ListedColormap(['#FF0000', '#00FF00'])

    #clf = KNeighborsClassifier(n_neighbors, weights=weights)
    #clf = LogisticRegression()
    clf.fit(X_mat, y_mat)

    # Plot the decision boundary by assigning a color in the color map
    # to each mesh point.
    
    mesh_step_size = .01  # step size in the mesh
    plot_symbol_size = 50
    
    x_min, x_max = X_mat[:, 0].min() - 1, X_mat[:, 0].max() + 1
    y_min, y_max = X_mat[:, 1].min() - 1, X_mat[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, mesh_step_size),
                         np.arange(y_min, y_max, mesh_step_size))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    ax.set_title('2 component PCA', fontsize = 20)
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot training points
    plt.scatter(X_mat[:, 0], X_mat[:, 1], s=plot_symbol_size, c=y, cmap=cmap_bold, edgecolor = 'black')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    print('xmin= %f, xman=%f; ymin=%f,ymax=%f'%(xx.min(), xx.max(), yy.min(), yy.max()))

    patch0 = mpatches.Patch(color='#FF0000', label='cancer')
    patch1 = mpatches.Patch(color='#00FF00', label='normal') 
    plt.legend(handles=[patch0, patch1])

        
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title('classification')
    
    plt.show()


In [ ]:
from sklearn.svm import SVC
from sklearn import svm
# knn = KNeighborsClassifier(n_neighbors=3, weights='uniform')
# logreg = LogisticRegression()
# lda = LinearDiscriminantAnalysis()
svm = svm.LinearSVC()

# plot_boundary(X_train, y_train, lda)

x = pca_todo.iloc[:, 0:35]
y = y_todo['class']

plot_boundary(x.values, y,svm)


## LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
ytr = y_train.loc[:,['class']].values.ravel()
yte = y_test.loc[:,['class']].values.ravel()
lda = LinearDiscriminantAnalysis() 
lda.fit(pca_train, y)
print('Accuracy of LDA classifier on training set: {:.2f}'.format(lda.score(pca_train, ytr)))
print('Accuracy of LDA classifier on test set: {:.2f}'.format(lda.score(pca_test, yte)))

In [ ]:
from sklearn.model_selection import cross_val_score
lda = LinearDiscriminantAnalysis()
scores = cross_val_score(lda,pca_todo.iloc[:, 0:35].values, y_todo.loc[:,['class']].values.ravel(), cv=10)
print("\nLDA")
print("Cross validation scores: {}".format(scores))
print("Average cross-validation score: %.2f +/- %.2f"%(scores.mean(), scores.std())) 

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = lda.predict(pca_test)
print(confusion_matrix(pca_test, pred))
print(classification_report(y_test, pred))

## GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(pca_train, y_train)
print('Accuracy of GNB classifier on training set: {:.2f}'.format(gnb.score(pca_train, y_train)))
print('Accuracy of GNB classifier on test set: {:.2f}'.format(gnb.score(pca_test, y_test)))

## Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='poly')
print(svm)
svm.fit(pca_train, y_train)
print('Accuracy of SVM classifier on training set: {:.2f}'.format(svm.score(pca_train, y_train)))
print('Accuracy of SVM classifier on test set: {:.2f}'.format(svm.score(pca_test, y_test)))

## Knn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'.format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'.format(knn.score(X_test, y_test)))

In [ ]:
from sklearn.model_selection import cross_val_score 
scores = cross_val_score(knn, X_todo, y, cv=10)
print("\nSupport vector machine")
print("Cross validation scores: {}".format(scores))
print("Average cross-validation score: %.2f +/- %.2f"%(scores.mean(), scores.std())) 

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = knn.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))

In [ ]:
# Logistic Regression
from sklearn.model_selection import cross_val_score
scores = cross_val_score(logreg, X_train, y_train, cv=10)
print("\nLogistic Regression")
print("Cross validation scores: {}".format(scores))
print("Average cross-validation score: %.2f +/- %.2f"%(scores.mean(), scores.std())) 

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier().fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'.format(dtree.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(dtree.score(X_test, y_test)))

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(dtree, X_train, y_train, cv=10)
print("\nDecision Tree")
print("Cross validation scores: {}".format(scores))
print("Average cross-validation score: %.2f +/- %.2f"%(scores.mean(), scores.std())) 

## Support vector machine

In [ ]:
from sklearn import svm


# svm = svm.SVC(gamma='scale')
svm = svm.LinearSVC()
# svm = svm.SVC(kernel='poly')
# svm = svm.SVR()
svm.fit(pca_train, y_train)  
print('Accuracy of SVM classifier on training set: {:.2f}'.format(svm.score(pca_train, y_train)))
print('Accuracy of SVM classifier on test set: {:.2f}'.format(svm.score(pca_test, y_test)))

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(svm, X_train, y_train, cv=10)
print("\nSupport vector machine")
print("Cross validation scores: {}".format(scores))
print("Average cross-validation score: %.2f +/- %.2f"%(scores.mean(), scores.std())) 


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = svm.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))